Tool Calling → lets the LLM decide which tool to use and what input to pass, based on the user’s natural language.

We use it because:

LLMs can’t fetch real-time data or run actual code.

Without it, programmers would need to manually pick and run tools.

With it, the system becomes dynamic, flexible, and natural.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
import requests
from langchain_core.tools import tool

d:\LangChain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()

True

## Too Creation

In [ ]:
# tool create
@tool
def multiply(a : int, b : int) -> int:
    """given 2 numbers a and b this tool returns their product"""
    
    return a * b

In [ ]:
print(multiply.invoke({'a' : 4, 'b' : 5}))

20


In [ ]:
multiply.name

'multiply'

In [ ]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [ ]:
multiply.description

'given 2 numbers a and b this tool returns their product'

## Tool Binding

In [ ]:
# tool binding
llm = ChatGoogleGenerativeAI(model = 'gemini-2.5-flash')

In [ ]:
llm_with_tools = llm.bind_tools([multiply])

In [ ]:
"""There are not all llm are allowed to bind the tools.
   some of llm's are allowed to bind the tools."""

"There are not all llm are allowed to bind the tools.\n   some of llm's are allowed to bind the tools."

## Tool Calling

LLM’s role:
It looks at the user’s prompt, decides which tool is most relevant, and figures out what input should be sent to that tool.

Tool’s role:
The tool actually performs the task (e.g., multiplying numbers, fetching weather, querying a database).

Final step:
The tool returns the result → LLM takes that raw result and explains it back to the user in natural language.

In [ ]:
# now we do that how and when a llm call a tool. This process is called tool calling.

In [ ]:
llm_with_tools.invoke("hi how are you")
# in this line we see a content. but not see the tool_call.

AIMessage(content="I'm doing well, thank you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--af502758-6ffc-42eb-9fcb-9c8d4fcf7866-0', usage_metadata={'input_tokens': 58, 'output_tokens': 16, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}})

In [ ]:
llm_with_tools.invoke("can you multiply 3 with 10.")
# in this lien we see the tool_Call but not see the content.

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--c582c084-6d8a-4e8f-8c59-427595f7c0bf-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 10.0}, 'id': '36051a22-54e0-4caa-a1d8-89fce15c0f2b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 64, 'output_tokens': 19, 'total_tokens': 134, 'input_token_details': {'cache_read': 0}})

In [ ]:
querry = HumanMessage("can you multiply 3 with 10")

In [ ]:
message = [querry]

In [ ]:
result = llm_with_tools.invoke(message)

In [ ]:
message.append(result)

## TOOL EXECUTION

Programmer can manually pass them(input)

Tool Calling = LLM chooses tool + sends inputs

Tool Execution = Programmer sends inputs manually

In [ ]:
tool_result = multiply.invoke(result.tool_calls[0]['args'])

In [ ]:
message.append(tool_result)

In [ ]:
message

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--515ac99e-60fd-4326-a05c-4d47c96e28e9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 10.0}, 'id': '8df1b657-1df9-4cc9-993c-b98830b4afb2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 19, 'total_tokens': 133, 'input_token_details': {'cache_read': 0}}),
 30]

In [ ]:
print(type(message), message)

<class 'list'> [HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--515ac99e-60fd-4326-a05c-4d47c96e28e9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 10.0}, 'id': '8df1b657-1df9-4cc9-993c-b98830b4afb2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 19, 'total_tokens': 133, 'input_token_details': {'cache_read': 0}}), 30]


In [ ]:
clean_message = [msg for msg in message if not isinstance(msg, int)]

response = llm_with_tools.invoke(clean_message)

In [ ]:
print(response.content)